# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization

[nltk_data] Downloading package punkt to /home/oliver/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/oliver/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/oliver/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# load data from database
engine = create_engine('sqlite:///../../DisasterResponse.db')
df = pd.read_sql('messages', con=engine)
X = df[['message']]
Y = df.iloc[:,4:]
Y.head()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """Tokenize the provided text by
        * removing punctuation
        * tokenizing into words
        * removing stopwords
        * lemmatizing, including conversion to lowercase
    """
    
    # remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # tokenize into words
    tokens = word_tokenize(text)
    
    # remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    
    # lemmatize, including conversion to lowercase
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
X.head()

,message
0,Weather update - a cold front from Cuba that c...
1,Is the Hurricane over or is it not over
2,Looking for someone but no name
3,UN reports Leogane 80-90 destroyed. Only Hospi...
4,"says: west side of Haiti, rest of the country ..."


In [7]:
# perform train test split
n_samples = X.shape[0]
X_train, X_test, Y_train, Y_test = train_test_split(X.message.iloc[:n_samples], Y.iloc[:n_samples], test_size=0.25)

In [8]:
# vectorize
# Thinkpad: 1:05 min.
vect = CountVectorizer(tokenizer=tokenize)
%time X_train_vect  = vect.fit_transform(X_train)

CPU times: user 1min, sys: 5.29 s, total: 1min 5s
Wall time: 1min 6s


In [9]:
# tfidf
tfidf = TfidfTransformer()
%time X_train_tfidf = tfidf.fit_transform(X_train_vect)

CPU times: user 58.2 ms, sys: 3.84 ms, total: 62 ms
Wall time: 1.05 s


In [10]:
# Train classifier
# Thinkpad, 1 core : 5:57 min.
# Thinkpad, 2 cores: 3:56 min.
# Thinkpad, 4 cores: 3:36 min.
clf = MultiOutputClassifier(RandomForestClassifier(), n_jobs=4)
%time clf.fit(X_train_tfidf, Y_train)

CPU times: user 1.82 s, sys: 1.58 s, total: 3.39 s
Wall time: 3min 36s


MultiOutputClassifier(estimator=RandomForestClassifier(), n_jobs=4)

In [11]:
# vectorize test data
%time X_test_vect  = vect.transform(X_test)

CPU times: user 19.5 s, sys: 1.74 s, total: 21.2 s
Wall time: 21.3 s


In [12]:
# tfidf test data
%time X_test_tfidf = tfidf.transform(X_test_vect)

CPU times: user 10.6 ms, sys: 441 µs, total: 11 ms
Wall time: 9.85 ms


In [13]:
# predict data
%time Y_pred = clf.predict(X_test_tfidf)

CPU times: user 2.5 s, sys: 662 ms, total: 3.16 s
Wall time: 9.65 s


In [14]:
Y_test

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
4499,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18386,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22755,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8802,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23403,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12799,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
12518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1230,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
Y

In [135]:
(Y == 1).sum(1)

0        0
1        4
2        0
3        7
4        0
        ..
26175    0
26176    0
26177    0
26178    2
26179    0
Length: 26180, dtype: int64

In [129]:
np.where(Y_pred == 1)

(array([   0,    4,    4, ..., 6539, 6544, 6544]),
 array([ 2,  0,  2, ...,  9, 27, 29]))

In [128]:
confusion_matrix(Y_test, Y_pred, labels=Y_test.columns)

ValueError: multilabel-indicator is not supported

In [ ]:
pipeline = 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.